## DASH VERSION 2 COMPLIQUER

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import pandas as pd
import plotly.express as px


df = pd.read_csv(r"C:\Users\dell\OneDrive\Desktop\cmc\2eme\visuilisation yassin\New folder\gapminder_unfiltered.csv")

unique_countries = df['country'].drop_duplicates().reset_index(drop=True)


app = Dash()

app.layout = html.Div([
    html.H1(children='Filtre Table et Graph', style={
        'color': 'black',
        'textAlign': 'center',
        'background-color': 'lightgray',
        'padding': '10px',
        'border-radius': '10px'
    }),

    dcc.Dropdown(
        options=[{'label': country, 'value': country} for country in unique_countries],
        value='Canada',
        id='dropdown_id',
        style={'width': '50%', 'margin': '20px auto'}
    ),

    
    dcc.Graph(figure={}, id='graph_id', style={'margin-bottom': '30px'}),

    dash_table.DataTable(
        id='table_id',
        columns=[{'name': col, 'id': col} for col in df.columns],  
        data=df.head(10).to_dict('records'),  
        style_table={'margin': '0 auto', 'width': '80%'},
        style_cell={'textAlign': 'center'},
        page_size=10,  
    ),


    html.Div([
        html.Button('Précédent', id='prev_button', n_clicks=0, style={'margin-right': '10px'}),
        html.Button('Suivant', id='next_button', n_clicks=0),
    ], style={'textAlign': 'center', 'margin-bottom': '20px'}),

])


@callback(
    Output(component_id='graph_id', component_property='figure'),
    Input(component_id='dropdown_id', component_property='value')
)
def update_graph(selected_country):
   
    filtered_data = df[df['country'] == selected_country]
   
    fig = px.line(filtered_data, x='year', y='lifeExp', title=f'Évolution de l’espérance de vie pour {selected_country}')
    return fig

@callback(
    Output(component_id='table_id', component_property='data'),
    Input('prev_button', 'n_clicks'),
    Input('next_button', 'n_clicks')
)
def update_table(prev_clicks, next_clicks):
    group_size = 10  
    current_page = max(0, next_clicks - prev_clicks)  
    start_idx = current_page * group_size  
    end_idx = start_idx + group_size  

    table_data = df.iloc[start_idx:end_idx]
    return table_data.to_dict('records')

if __name__ == "__main__":
    app.run_server(debug=True, port=8060)